# Data Extract, Transform, Load

This notebook objective is to scrape 2 subreddit websites, followed by transforming the unstructured data through 2 different prepocessing techniques, and export the cleaned and structured data for exploratory analysis.  

## Problem Statement

Google's smart speaker system, Google Home, was designed to compete with the popular Amazon Echo. Both product serve as a vehicle to their respective voice-activated virtual helper that connects to the internet. 

Reddit users have used the platform as a forum to discuss their experience with the products. I had been tasked by Google's Research team to analyze customer sentiment towards Google Home from subreddit posts on 'r/GoogleHome'.

Additionally the Research Team would also like to find out what common and unique customer pain points are prevalent between the Google Home and Amazon Echo, with the goal of designing a better product. Therefore subreddit posts from 'r/AmazonEcho' would also be included in the dataset.

A Random Forrest model would also be built to predict if a given set of words do in fact refer to the discussion of either the Amazon Echo or the Google Home based on selected features. 

Each subreddit post are represented as 'documents' in the dataset and therefore both terms will be used interchangebly.


**Contents**
- [Import libraries](#Import-libraries)
- [Data collection via webscrapping](#Data-collection-via-webscrapping)
- [Data cleaning](#Data-cleaning)
    - [investigate on documents with missing selftext](#investigate-on-documents-with-missing-selftext)
    - [Investigate for reoccuring urls](#Investigate-for-reoccuring-urls)
    - [Investigate posts made by same authors](#Investigate-posts-made-by-same-authors)
    - [Dropping unwanted documents](#Dropping-unwanted-documents)
- [Feature Selction and Engineering](#Feature-Selction-and-Engineering)

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import re

## Data collection via webscrapping
#### *Only run notebook code in the next section. This section is to only illustrate webscrapping codeblocks.

In [2]:
# """Using Push Shift api to webscrape
# through subreddit r/GoogleHome and r/AmazonEcho"""

# assign url for request
# url =  'https://api.pushshift.io/reddit/search/submission'

In [3]:
# """using a while loop to iterate 
# through posts from different utc values
# and break when the dataframe has reached 
# desired length"""

# ggl_params = {
#     'subreddit': 'googlehome',
#     'size': 100,
#     'before': None
# }

# ggl_df = pd.DataFrame()   #created empty df to be concated to and while loop bool

# while len(ggl_df) < 3000:
#     res = requests.get(url,ggl_params)              #create request
#     data = res.json()['data']                       #extract j.son data
#     data = pd.DataFrame(data)                       #create extrated data into df
#     ggl_df = pd.concat([ggl_df,data],axis=0)         # concat new df with old df
#     ggl_df.reset_index(drop=True,inplace=True)          #reset df index
#     ggl_params['before'] = ggl_df.created_utc[len(ggl_df)-1] #change params
    
    
# print(ggl_df.shape)


In [4]:
# """using a while loop to iterate 
# through posts from different utc values
# and break when the dataframe has reached 
# desired length"""

# amz_params = {
#     'subreddit': 'amazonecho',
#     'size': 100,
#     'before': None
# }

# amz_df = pd.DataFrame()   #created empty df to be concated to and while loop bool

# while len(amz_df) < 3000:
#     res = requests.get(url,amz_params)              #create request
#     data = res.json()['data']                       #extract j.son data
#     data = pd.DataFrame(data)                       #create extrated data into df
#     amz_df = pd.concat([amz_df,data],axis=0)         # concat new df with old df
#     amz_df.reset_index(drop=True,inplace=True)          #reset df index
#     amz_params['before'] = amz_df.created_utc[len(amz_df)-1] #change params
    
    
# print(amz_df.shape)

In [5]:
# export unstructured data
# ggl_df.to_csv(r'../datasets/unclean_google_data.csv')
# amz_df.to_csv(r'../datasets/unclean_amazon_data.csv')

## Data cleaning
#### *run code starting from this section of notebook

In [6]:
ggl_df = pd.read_csv('../datasets/unclean_google_data.csv')
amz_df = pd.read_csv('../datasets/unclean_amazon_data.csv')

In [7]:
#check shape of dfs

print(f'Google Home df shape:{ggl_df.shape}')
print(f'Amazon Echo df shape:{amz_df.shape}')    

Google Home df shape:(3000, 85)
Amazon Echo df shape:(3000, 83)


In [8]:
"""sanity check to see if scraping was done correctly
through checking of dupplicates using title and utc"""

print(f"number of duplicates in Goggle Home df: {ggl_df[ggl_df.duplicated(subset=('title','created_utc'))].shape[0]}")

print(f"number of duplicates in Amazon echo df: {amz_df[amz_df.duplicated(subset=('title','created_utc'))].shape[0]}")

number of duplicates in Goggle Home df: 0
number of duplicates in Amazon echo df: 0


In [9]:
#secondary sanity check for duplicates on self text

print(f"number of duplicates for self text in Goggle Home df: {ggl_df[ggl_df.duplicated(subset = 'selftext')].shape[0]}")

print(f"number of duplicates for self text in Amazon echo df: {amz_df[amz_df.duplicated(subset='selftext')].shape[0]}")

number of duplicates for self text in Goggle Home df: 918
number of duplicates for self text in Amazon echo df: 765


In [10]:
#further investigation of Google Home df self.text duplicates

ggl_df[ggl_df.duplicated(subset = 'selftext')]['selftext'].head()

3     NaN
14    NaN
28    NaN
38    NaN
43    NaN
Name: selftext, dtype: object

In [11]:
#Check to see corresponding titles of documents with empty self text

ggl_df[ggl_df.duplicated(subset = 'selftext')]['title'].head()

3            Trouble enabling Voice Match for 2nd user?
14    Project CHIP Finally coming end of 2021, wonde...
28    Samsung Range not able to connect to Google Ho...
38    Buy Google 5 Star Reviews - 100% Safe &amp; Pe...
43    How to Trigger Routines at Sunrise/Sunset on G...
Name: title, dtype: object

In [12]:
#repeat same steps with Amazon Echo df

amz_df[amz_df.duplicated(subset = 'selftext')]['selftext'].head()

13    NaN
17    NaN
20    NaN
27    NaN
30    NaN
Name: selftext, dtype: object

In [13]:
#Check to see corresponding titles of documents with empty self text

amz_df[amz_df.duplicated(subset = 'selftext')]['title'].head()

13                                                Day 9
17    $20 off All-new Echo Buds (2nd Gen) for pre-or...
20    23% off Echo Show 8 HD smart display with Alex...
27    My wife prefers a specific temperature inside ...
30      Amazon announces new Echo Buds with ANC feature
Name: title, dtype: object

<b>Observation:</b><br/>
917 documents in the Ggl_df and 765 documents in Amz_df are posts that had titles but no self text. 

Potential quesitons to answer are as follows:

Are posts that are solely headings useful data?<br/>
Should posts that are solely titles be feature engineered as self text?<br/>
Do these posts tend to include comments, or URL or other similarities?<br/>

We want to ensure that these post would not negatively affect the predictive power of our model.


### Investigate documents with missing selftext

In [14]:
#create new dfs of documents with only titles to investigate usefulness

ggl_titles_only = ggl_df[ggl_df.duplicated(subset = 'selftext')].index.tolist() #put index of documents with titles only into list
ggl_title_only_df = ggl_df.iloc[ggl_titles_only,:]    #create df of documents with titles only

amz_titles_only = amz_df[amz_df.duplicated(subset = 'selftext')].index.tolist() #put index of documents with titles only into list
amz_title_only_df = amz_df.iloc[amz_titles_only,:]    #create df of documents with titles only


In [15]:
"""refering to the data dectionary extracted from the j.son format of the subreddit websites,
post are also categorized based on the type of posts, 'link_flair_text'"""

print(ggl_title_only_df.link_flair_text.value_counts())
print('\n')
print(f'Number of Ggl documents that are solely titles without a link_flair_text category: {len(ggl_title_only_df)-ggl_title_only_df.link_flair_text.value_counts().sum()}')

Help                           125
Bug                             43
Other                           40
Tips                            20
News                            18
Hacks                            8
NSFW - Language                  8
Features WishList                8
Product Review                   7
Deals | Sales | Promotions       7
Commands | How To's              7
GH Trigger Warning               3
Name: link_flair_text, dtype: int64


Number of Ggl documents that are solely titles without a link_flair_text category: 624


In [16]:
print(amz_title_only_df.link_flair_text.value_counts())
print('\n')
print(f'Number of Amz documents that are solely titles without a link_flair_text category: {len(amz_title_only_df)-(amz_title_only_df.link_flair_text.value_counts()).sum()}')

Question           158
Review              72
Technical Issue     63
Feature             32
Alexa Skill         31
Feature Request     18
Easter Egg           2
Skill Request        1
Name: link_flair_text, dtype: int64


Number of Amz documents that are solely titles without a link_flair_text category: 388


Inference:

More than half of each subreddit dataframe has a link_flair_text category. Based on their category name, for example 'Question' and 'Bug', are more than likely to be posts that authors feel it would be redundant to elaborate in 'selftext', as just the titles would be self explanatory or instead of text, an image or url was used. 

These post would also likely be requesting for assistance or interaction in the form of comments. this inference can be supported by comparing the distrubution of comments or the presence of an image.


#### Check distribution of comments

In [17]:
#check the distribution of comments for only headings documents with a link_flair_text category

ggl_title_only_df[ggl_title_only_df.link_flair_text.notnull()].num_comments.describe()

count    294.000000
mean      10.125850
std       22.619296
min        0.000000
25%        0.000000
50%        2.000000
75%        9.000000
max      185.000000
Name: num_comments, dtype: float64

<b>observation</b>:
Average of comments is 10 per document however this is likely skewed from a post with 185 comments. Further investigation is needed on this document.

In [18]:
# show data dictionary of document with 185 comments

ggl_title_only_df[ggl_title_only_df.num_comments == 185].to_dict()

{'Unnamed: 0': {1242: 1242},
 'all_awardings': {1242: "[{'award_sub_type': 'GLOBAL', 'award_type': 'global', 'awardings_required_to_grant_benefits': None, 'coin_price': 80, 'coin_reward': 0, 'count': 1, 'days_of_drip_extension': 0, 'days_of_premium': 0, 'description': 'Everything is better with a good hug', 'end_date': None, 'giver_coin_reward': 0, 'icon_format': 'PNG', 'icon_height': 2048, 'icon_url': 'https://i.redd.it/award_images/t5_q0gj4/ks45ij6w05f61_oldHugz.png', 'icon_width': 2048, 'id': 'award_8352bdff-3e03-4189-8a08-82501dd8f835', 'is_enabled': True, 'is_new': False, 'name': 'Hugz', 'penny_donate': 0, 'penny_price': 0, 'resized_icons': [{'height': 16, 'url': 'https://preview.redd.it/award_images/t5_q0gj4/ks45ij6w05f61_oldHugz.png?width=16&amp;height=16&amp;auto=webp&amp;s=73a23bf7f08b633508dedf457f2704c522b94a04', 'width': 16}, {'height': 32, 'url': 'https://preview.redd.it/award_images/t5_q0gj4/ks45ij6w05f61_oldHugz.png?width=32&amp;height=32&amp;auto=webp&amp;s=50f2f16e71d2

In [19]:
#print url of post for further investigation
print (f"Link to title only post, with a link_flair_text category, with highest number of comments: {ggl_title_only_df[ggl_title_only_df.num_comments == 185].to_dict()['full_link'][1242]}")

Link to title only post, with a link_flair_text category, with highest number of comments: https://www.reddit.com/r/googlehome/comments/lrvlop/finally_found_one_of_those_crazy_target_deals_had/


<b>observation</b>: Upon further investigation the document, this post had gone 'viral', an outlier occurence where many other redittors had interacted with this post.
    
The title of this post being 'Finally found one of those crazy target deals! Had to pick them up'. Followed by an image of a receipt where the redditor had purchased the Google Home at a low price.
    
This is also a good example of how just the heading alone conveys the overall sentiment of what the redditors wants to post without selftext.

In [20]:
#check the distribution of comments for only headings documents without a link_flair_text category

"""use link_flair_text.isnull() for documents without link_flair_text.isnull()""" 
ggl_title_only_df[ggl_title_only_df.link_flair_text.isnull()].num_comments.describe()

count    624.000000
mean       5.190705
std       19.605021
min        0.000000
25%        0.000000
50%        0.000000
75%        2.000000
max      221.000000
Name: num_comments, dtype: float64

In [21]:
print(f'Google Home df shape:{ggl_df.shape}')
print(f'Amazon Echo df shape:{amz_df.shape}')

Google Home df shape:(3000, 85)
Amazon Echo df shape:(3000, 83)


<b>observation</b>:
Average of comments is 5 per document however this is likely skewed from a post with 221 comments. Further investigation is needed on this document. The median and 25 percentile is 0 highlighting barely any interaction.

In [22]:
# show data dictionary of document with 221 comments

ggl_title_only_df[ggl_title_only_df.num_comments == 221].to_dict()

{'Unnamed: 0': {1206: 1206},
 'all_awardings': {1206: "[{'award_sub_type': 'GLOBAL', 'award_type': 'global', 'awardings_required_to_grant_benefits': None, 'coin_price': 125, 'coin_reward': 0, 'count': 1, 'days_of_drip_extension': 0, 'days_of_premium': 0, 'description': 'When you come across a feel-good thing.', 'end_date': None, 'giver_coin_reward': None, 'icon_format': None, 'icon_height': 2048, 'icon_url': 'https://i.redd.it/award_images/t5_22cerq/5izbv4fn0md41_Wholesome.png', 'icon_width': 2048, 'id': 'award_5f123e3d-4f48-42f4-9c11-e98b566d5897', 'is_enabled': True, 'is_new': False, 'name': 'Wholesome', 'penny_donate': None, 'penny_price': None, 'resized_icons': [{'height': 16, 'url': 'https://preview.redd.it/award_images/t5_22cerq/5izbv4fn0md41_Wholesome.png?width=16&amp;height=16&amp;auto=webp&amp;s=92932f465d58e4c16b12b6eac4ca07d27e3d11c0', 'width': 16}, {'height': 32, 'url': 'https://preview.redd.it/award_images/t5_22cerq/5izbv4fn0md41_Wholesome.png?width=32&amp;height=32&amp;au

In [23]:
#print url of post for further investigation
print (f"Link to title only post without a link_flair_text category, with highest number of comments: {ggl_title_only_df[ggl_title_only_df.num_comments == 221].to_dict()['full_link'][1206]}")

Link to title only post without a link_flair_text category, with highest number of comments: https://www.reddit.com/r/googlehome/comments/lsjp5n/google_kills_most_of_its_disney_readalong_books/


<b>observation</b>: Similar to the document which had high number of comments, this document had also gone 'viral'.
    
The title of this post being 'Google kills most of its Disney read-along books for Nest and Home'. This is news relating the product. There were no images and only a url.
    
This is another a good example of how just the heading alone conveys the overall sentiment of what the redditors wants to post without self.text.

<b>Conclusion</b>: from the above, it is safe to assume that for posts not having selftext, as well as a link_flair_text category, the titles more or less captures the sentiment or overall message that the redditor is espressing. Albeit some post could be peculiar or nonsensical yet still having something to do with the product.
    
Before deciding to transform documents with only titles (copying the words of the respective title to it being under the self.text column), I would like to know what is the frequency of the presense of a url or photo for these posts. This can be determine by the feature 'url_overridden_by_dest'. The presence of an image would have a value of "https://i.imgur.com..." whereas a link would have the respective link itself. 

#### Gauge percentage of title only documents with links or pictures

In [24]:
ggl_title_only_df.groupby(by='link_flair_text')['url_overridden_by_dest'].count()

link_flair_text
Bug                            38
Commands | How To's             5
Deals | Sales | Promotions      7
Features WishList               6
GH Trigger Warning              2
Hacks                           7
Help                           90
NSFW - Language                 7
News                           16
Other                          36
Product Review                  5
Tips                           16
Name: url_overridden_by_dest, dtype: int64

In [25]:
"""Use group by function to only show presence of 
url or image per link_flair_text category on Google Home df"""

#count of post with url divide by total number of posts per link_flair_text
ggl_title_only_df.groupby(by='link_flair_text')['url_overridden_by_dest'].count()/ggl_title_only_df.groupby(by='link_flair_text')['url_overridden_by_dest'].agg(lambda x: np.count_nonzero(x))

link_flair_text
Bug                            0.883721
Commands | How To's            0.714286
Deals | Sales | Promotions     1.000000
Features WishList              0.750000
GH Trigger Warning             0.666667
Hacks                          0.875000
Help                           0.720000
NSFW - Language                0.875000
News                           0.888889
Other                          0.900000
Product Review                 0.714286
Tips                           0.800000
Name: url_overridden_by_dest, dtype: float64

In [26]:
print(f'Percentage of Google Home posts without selftext that has a url or image: {ggl_title_only_df.url_overridden_by_dest.count()/len(ggl_title_only_df)*100}%')

Percentage of Google Home posts without selftext that has a url or image: 84.74945533769062%


In [27]:
"""Use group by function to only show presence of 
url or image per link_flair_text category on Amazon df"""

#count of post with url divide by total number of posts per link_flair_text
amz_title_only_df.groupby(by='link_flair_text')['url_overridden_by_dest'].count()/amz_title_only_df.groupby(by='link_flair_text')['url_overridden_by_dest'].agg(lambda x: np.count_nonzero(x))

link_flair_text
Alexa Skill        0.741935
Easter Egg         0.000000
Feature            0.906250
Feature Request    0.500000
Question           0.430380
Review             0.805556
Skill Request      0.000000
Technical Issue    0.619048
Name: url_overridden_by_dest, dtype: float64

In [28]:
print(f'Percentage of Amazon Echo posts without self.text that has a url or image: {amz_title_only_df.url_overridden_by_dest.count()/len(amz_title_only_df)*100}%')

Percentage of Amazon Echo posts without self.text that has a url or image: 61.568627450980394%


<b>observation</b>: From the above we can see that more than 50 percent posts from both subreddits that had no selftext and only a title, had included either a link or an image. This confirmed my earlier assumption. 

Discovering this has prompted me to look further into the documents. I am curious to find out if there are any reoccuring urls, likely from authors promoting something. If so, these documents would have to be looked at and assessed if their posts are meaningful to our model, otherwise they would have to be removed. 

### Investigate for reoccuring urls

In [29]:
#check for top 10 common urls in Google Home ddf
ggl_df.url_overridden_by_dest.value_counts().sort_values(ascending=False).head(10)

https://reviewsfund.com/product/buy-facebook-reviews/                              54
https://reviewsfund.com/product/buy-trustpilot-reviews/                            53
https://usareviewshop.com/product/buy-verified-paypal-accounts/                    46
https://usareviewshop.com/product/buy-verified-cash-app-accounts/                  37
https://usatopservices.com/product/buy-google-5-star-reviews/                      13
https://usatopservices.com/product/buy-facebook-page-likes/                        10
https://reviewsfund.com/product/buy-sitejabber-reviews/                             7
https://tiktokvpnforindia.blogspot.com/2021/02/android-best-ripple-vpn-app.html     5
https://usatopsmm.com/product/buy-stripe-account/                                   4
https://www.reddit.com/                                                             4
Name: url_overridden_by_dest, dtype: int64

In [30]:
#check title,selftext and authors of posts with reoccuring urls for further investigation

common_ggl_url = ggl_df.url_overridden_by_dest.value_counts().head(10).index.to_list()

#use for loop to iterate through different attributes
for x in common_ggl_url:
    print (f'Findings on posts with url: {x}')
    print (f"Number of posts with url: {len(ggl_df[ggl_df.url_overridden_by_dest == x])}")
    print(f"Total number of selftext: {ggl_df[ggl_df.url_overridden_by_dest == x]['selftext'].sum()}")
    print(f"Unique titles:")
    print(ggl_df[ggl_df.url_overridden_by_dest == x]['title'].unique())
    print(f"Total unique authors: {len(ggl_df[ggl_df.url_overridden_by_dest == x]['author'].values)}")
    print('\n')

Findings on posts with url: https://reviewsfund.com/product/buy-facebook-reviews/
Number of posts with url: 54
Total number of selftext: 0
Unique titles:
['Buy Facebook Reviews - USA Facebook Page Positive Reviews Rating']
Total unique authors: 54


Findings on posts with url: https://reviewsfund.com/product/buy-trustpilot-reviews/
Number of posts with url: 53
Total number of selftext: 0
Unique titles:
['Buy Trustpilot Reviews - 100% USA, UK, AU Permanent Trustpilot Reviews'
 'Buy Trustpilot Reviews - 100% USA, UK, AU Permanen Trustpilot Reviews'
 'Buy Trustpilot Reviews - 100% Permanent USA, UK, AU Reviews']
Total unique authors: 53


Findings on posts with url: https://usareviewshop.com/product/buy-verified-paypal-accounts/
Number of posts with url: 46
Total number of selftext: 0
Unique titles:
['Buy Verified PayPal Accounts - USA Verified Personal &amp; Business Account']
Total unique authors: 46


Findings on posts with url: https://usareviewshop.com/product/buy-verified-cash-app-a

In [31]:
#check for top 10 common urls in Amazon Echo ddf
amz_df.url_overridden_by_dest.value_counts().sort_values(ascending=False).head(10)

https://digitalservice24h.com/service/buy-amazon-accounts/                                 9
https://cpavox.com/track/0463fe4fb8e                                                       7
http://worldquizzes.com                                                                    3
http://sweepstakes2021.com                                                                 2
https://zasnewsshshidri.blogspot.com/2020/12/sense-energy-monitor-with-solar-track.html    2
https://direct-link.net/221756/activation                                                  2
https://www.thesecretline.xyz/2021/01/amazon-offering-up-to-56-off-on-several.html         2
http://worldinfo.tn/molicui-vertical-charging-stand-compatible-with-sony-ps5/              2
http://smarthome.soulsprawl.com/find-out-products-what-you/#smarthome                      2
https://www.reddit.com/gallery/kx58d2                                                      1
Name: url_overridden_by_dest, dtype: int64

In [32]:
#check title, selftext and authors of posts with reoccuring urls for further investigation

common_amz_url = amz_df.url_overridden_by_dest.value_counts().sort_values(ascending=False).head(10).index.to_list()

#use for loop to iterate through different attributes
for x in common_amz_url:
    print (f'Findings on posts with url: {x}')
    print (f"Number of posts with url: {len(amz_df[amz_df.url_overridden_by_dest == x])}")
    print(f"Total number of selftext: {amz_df[amz_df.url_overridden_by_dest == x]['selftext'].sum()}")
    print(f"Unique titles:")
    print(amz_df[amz_df.url_overridden_by_dest == x]['title'].unique())
    print(f"Total unique authors: {len(amz_df[amz_df.url_overridden_by_dest == x]['author'].values)}")
    print('\n')

Findings on posts with url: https://digitalservice24h.com/service/buy-amazon-accounts/
Number of posts with url: 9
Total number of selftext: 0
Unique titles:
['Buy Amazon Accounts' 'Buy Amazon Accounts - Buy Old Amazon Accounts'
 'Buy Amazon Accounts - Buy Email Verified Amazon Accounts']
Total unique authors: 9


Findings on posts with url: https://cpavox.com/track/0463fe4fb8e
Number of posts with url: 7
Total number of selftext: 0
Unique titles:
['Earn Free Amazon Gift Cards &amp; Codes Legally - GiftsJunkie']
Total unique authors: 7


Findings on posts with url: http://worldquizzes.com
Number of posts with url: 3
Total number of selftext: 0
Unique titles:
['All-new Echo Show 10 (3rd Gen) , Buy 2, save $100 off - today on Amazon'
 "40% off All-new Echo Dot (4th Gen) + Amazon Smart Plug, today's deal"
 '68% off Outlet Wall Mount Solution for Echo Dot Gen 2 - on Amazon today']
Total unique authors: 3


Findings on posts with url: http://sweepstakes2021.com
Number of posts with url: 2
T

In [33]:
# find % of subreddit df that had reoccuring urls

print (f'{round(ggl_df.url_overridden_by_dest.value_counts().head(10).sum()/len(ggl_df)*100,2)}% of the Google Home subreddit had reocuring urls')
print (f'{round(amz_df.url_overridden_by_dest.value_counts().head(10).sum()/len(amz_df)*100,2)}% of the Amazon Echo subreddit had reocuring urls')

7.77% of the Google Home subreddit had reocuring urls
1.07% of the Amazon Echo subreddit had reocuring urls


<b>observation</b>: From the findings above, top 10 reoccuring urls posts, do not have selftext and some had variations of the same title. These post were all made by unique authors. Majority of these urls are promotional but are leaning towards the definition of 'spam' or even phishing sites. 

Another observation is that the Google Home subreddit had approximately 7 times more of these type of urls compared to the Amazon home subreddit from a sample of 3000 posts each.

Moving forward, these documents would be removed from their respective dataframes as they are infact not meaningful for out predictions but these findings are worth mentioning. 

In [34]:
print(f'Google Home df shape:{ggl_df.shape}')
print(f'Amazon Echo df shape:{amz_df.shape}')

Google Home df shape:(3000, 85)
Amazon Echo df shape:(3000, 83)


### Investigate posts made by same authors

Similar to uncovering interesting findings with reoccuring urls, this section will look into multiple posts made by the same authors

In [35]:
#show top 10 Google Home subreddit authors

ggl_df.author.value_counts().head(10)

[deleted]          31
halime123          27
oyirinnayaa        23
2tuff4u2            9
johnkhoo            9
monicakmtx          9
HeyCharrrrlie       7
DerrickWolfesmm     7
Newwales2           6
newyerker           6
Name: author, dtype: int64

In [36]:
# check '[deleted]' author against text title,and url

ggl_df[ggl_df.author == '[deleted]'][['selftext','title','url_overridden_by_dest']]

,selftext,title,url_overridden_by_dest
293,[deleted],"Where the fuck is the ""routines"" option in the...",NaN
301,[deleted],Just got a 2nd gen Nest Hub. A couple of thing...,NaN
303,[deleted],Removed Nest Mini from Home app but now I can'...,NaN
1063,[deleted],Emergency Calls from Nest Mini for Children,NaN
1069,[deleted],Help! I cannot connect to the Gosung app.,https://v.redd.it/35r1uqq97vk61
1151,[deleted],Leaked photos of the new Nest Hub Bastard,https://www.reddit.com/gallery/ltnubf
1172,NaN,Chromecast connection issue,NaN
1209,[deleted],New Google Nest Hub,NaN
1249,[deleted],"Google mini just announced ""someone is at your...",NaN
1283,[deleted],"Voice Match states ""Trouble Connecting"" when I...",NaN


In [37]:
#show top 10 Amazon Echo subreddit authors

amz_df.author.value_counts().head(10)

Kindly_Baby4695         28
[deleted]               25
Secure-Quality-6516     20
RamITT                  10
IfuDidntCome2Party      10
Legend1138               8
redwingshat              8
AXXXXXXXXA               8
_BindersFullOfWomen_     7
Bakura_1993              7
Name: author, dtype: int64

In [38]:
# check 'Kindly_baby4695' author against text,title and url

amz_df[amz_df.author == 'Kindly_Baby4695'][['selftext','title','url_overridden_by_dest']]

,selftext,title,url_overridden_by_dest
1006,NaN,The Rending and the Nest Hardcover,https://www.shahidriaz.com/2021/01/the-rending...
1437,NaN,Logitech G502 Hero High Performance Gaming Mouse,https://www.shahidriaz.com/2021/01/logitech-g5...
1451,NaN,"8"" LED Selfie Ring Light for Live Stream/Makeu...",https://www.shahidriaz.com/2021/01/8-led-selfi...
1452,NaN,Linenspa All-Season White Down Alternative Qui...,https://youtube.com/watch?v=MrikkijE-LA&amp;fe...
1455,NaN,Linenspa All-Season White Down Alternative Qui...,https://www.shahidriaz.com/2021/01/linenspa-al...
1457,NaN,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",https://www.shahidriaz.com/2021/01/acer-aspire...
1460,NaN,Sperry Instruments STK001 Non-Contact Voltage ...,https://www.shahidriaz.com/2021/01/sperry-inst...
1464,NaN,Apple AirPods with Charging Case (Wired),https://www.shahidriaz.com/2021/01/apple-airpo...
1486,NaN,"Belkin 12-Outlet Power Strip Surge Protector, ...",https://www.shahidriaz.com/2020/12/belkin-12-o...
1489,NaN,Etekcity Digital Body Weight Bathroom Scale wi...,https://youtube.com/watch?v=mac4KPW3e9A&amp;fe...


In [39]:
ggl_df.author.value_counts().sort_values(ascending=False).head(15)

[deleted]               31
halime123               27
oyirinnayaa             23
2tuff4u2                 9
johnkhoo                 9
monicakmtx               9
HeyCharrrrlie            7
DerrickWolfesmm          7
Regular_Raspberry_48     6
thirteen_20              6
PrestigiousBell8866      6
Newwales2                6
Kinglens311              6
newyerker                6
lukusw78                 5
Name: author, dtype: int64

In [40]:
#print findings of top 10 authors in Google Home subreddit

ggl_top_authors = ggl_df.author.sort_values(ascending=False).head(10).index.to_list() #make top authors into list

for x in ggl_top_authors:
    print (f'Author name: {x}')
    print (f"Number of posts: {len(ggl_df[ggl_df.author == x]['selftext'])}") #print number of posts
    print (f"Number of urls: {(ggl_df[ggl_df.author == x]['url_overridden_by_dest'].notnull()).sum()}") #print number of urls to compare with number of posts
    print (f"Percetage of urls to posts: {round(ggl_df[ggl_df.author == x]['url_overridden_by_dest'].notnull().sum()/len(ggl_df[ggl_df.author == x]['selftext'])*100,2)}%")
    print ('\n')


Author name: 855
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 385
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1398
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 2118
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1726
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 184
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 452
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1928
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1824
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 2169
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%




<ipython-input-40-25fb40a02a24>:9: RuntimeWarning: invalid value encountered in long_scalars
  print (f"Percetage of urls to posts: {round(ggl_df[ggl_df.author == x]['url_overridden_by_dest'].notnull().sum()/len(ggl_df[ggl_df.author == x]['selftext'])*100,2)}%")


In [41]:
print(ggl_top_authors)

[855, 385, 1398, 2118, 1726, 184, 452, 1928, 1824, 2169]


In [42]:
#print findings of top 10 authors in Amazon Echo subreddit

amz_top_authors = amz_df.author.sort_values(ascending=False).head(10).index.to_list() #make top authors into list

for x in amz_top_authors:
    print (f'Author name: {x}')
    print (f"Number of posts: {len(amz_df[amz_df.author == x]['selftext'])}") #print number of posts
    print (f"Number of urls: {amz_df[amz_df.author == x]['url_overridden_by_dest'].notnull().sum()}") #print number of urls to compare with number of posts
    print (f"Percetage of urls to posts: {round(amz_df[amz_df.author == x]['url_overridden_by_dest'].notnull().sum()/len(amz_df[amz_df.author == x]['selftext'])*100,2)}%")
    print ('\n')


Author name: 52
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1356
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 737
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 262
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 2490
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 720
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1786
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 2182
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 1883
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%


Author name: 589
Number of posts: 0
Number of urls: 0
Percetage of urls to posts: nan%




<ipython-input-42-686683d27424>:9: RuntimeWarning: invalid value encountered in long_scalars
  print (f"Percetage of urls to posts: {round(amz_df[amz_df.author == x]['url_overridden_by_dest'].notnull().sum()/len(amz_df[amz_df.author == x]['selftext'])*100,2)}%")


In [43]:
#create lists of authors to drop if % of urls to post > 0.5

# authors to drop in google df
ggl_authors_to_drop = []

for x in ggl_top_authors:
    
    if (ggl_df[ggl_df.author == x]['url_overridden_by_dest'].notnull().sum())/len(ggl_df[ggl_df.author == x]['selftext']) > 0.5:
        ggl_authors_to_drop.append(x)                                                                                                                                                              

# authors to drop in amazon df
amz_authors_to_drop = []

for x in amz_top_authors:
    if (amz_df[amz_df.author == x]['url_overridden_by_dest'].notnull().sum())/len(amz_df[amz_df.author == x]['selftext']) > 0.5:
        amz_authors_to_drop.append(x)
    

<ipython-input-43-7c92ae6e6f65>:8: RuntimeWarning: invalid value encountered in long_scalars
  if (ggl_df[ggl_df.author == x]['url_overridden_by_dest'].notnull().sum())/len(ggl_df[ggl_df.author == x]['selftext']) > 0.5:
<ipython-input-43-7c92ae6e6f65>:15: RuntimeWarning: invalid value encountered in long_scalars
  if (amz_df[amz_df.author == x]['url_overridden_by_dest'].notnull().sum())/len(amz_df[amz_df.author == x]['selftext']) > 0.5:


<b>observation</b>: Both subreddits had authors named '[deleted]' as authors with high number posts. when measured against number of urls to number of posts, they scored a low percentage. Based on their titles, majority are troubleshooting related. Perhaps these authors had deleted their reddit account or had been removed by reddit for a variety of reasons.

As for authors with high percentage of urls to post, these tend to be authors that are promoting a site or similar to the section above, spamming the subreddit.

Moving forward, I will drop authors with high urls per post percentage above 50%, amongst the top 10 authors with multiple posts to ensure that the models are not affected. 

### Dropping unwanted documents

In [44]:
#print dataframe shape prior to dropping unwanted data

print (f'Google Home dataset shape before removing unwanted data: {ggl_df.shape}')
print (f'Amazon dataset shape before removing unwanted data: {amz_df.shape}')

Google Home dataset shape before removing unwanted data: (3000, 85)
Amazon dataset shape before removing unwanted data: (3000, 83)


In [45]:
#drop unwanted reoccuring url from respective dataset by index

#create list of location index with reoccuring url in Google Home df
ggl_index_to_drop = ggl_df[ggl_df['url_overridden_by_dest'].isin(common_ggl_url)].index.to_list()


#create list of location index with authors to drop in Google Home df
ggl_author_index_to_drop = ggl_df[ggl_df['author'].isin(ggl_authors_to_drop)].index.to_list()


#create list of location index with reoccuring url in Amazon Echo df
amz_index_to_drop = amz_df[amz_df['url_overridden_by_dest'].isin(common_amz_url)].index.to_list()


#create list of location index with authors to drop in Amazon Echo df
amz_author_index_to_drop = amz_df[amz_df['author'].isin(amz_authors_to_drop)].index.to_list()


In [46]:
#Ensure there are no repeated index

#final rows to delete from Google Home df
ggl_index_to_drop_final = set(ggl_author_index_to_drop + ggl_index_to_drop)
print(f'Number of rows to be deleted from Google Home df: {len(ggl_index_to_drop_final)}')

#final rows to delete from Amazon Echo df
amz_index_to_drop_final = set(amz_author_index_to_drop + amz_index_to_drop)
print(f'Number of rows to be deleted from Amazon Echo df: {len(amz_index_to_drop_final)}')

Number of rows to be deleted from Google Home df: 233
Number of rows to be deleted from Amazon Echo df: 32


In [47]:
#Delete rows of unwanted data from respective df

#Delete rows off Google Home df
ggl_df.drop(index=ggl_index_to_drop_final, inplace=True)
ggl_df.reset_index(inplace=True, drop=True) #reset df index and drop old index column

#Delete rows off Amazon Echo df
amz_df.drop(index=amz_index_to_drop_final, inplace=True)
amz_df.reset_index(inplace=True, drop=True) #reset df index and drop old index column


print (f'Google Home dataset shape after removing unwanted data: {ggl_df.shape}')
print (f'Amazon dataset shape after removing unwanted data: {amz_df.shape}')

Google Home dataset shape after removing unwanted data: (2767, 85)
Amazon dataset shape after removing unwanted data: (2968, 83)


In [48]:
#concat subreddit df into on df

joined_df = pd.concat([ggl_df,amz_df],axis=0)
joined_df.reset_index(drop=True,inplace=True)

print(f'Combined dataframe shape: {joined_df.shape}')

Combined dataframe shape: (5735, 87)


## Feature Selction and Engineering

In [49]:
#show columns of new df
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 87 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     5735 non-null   int64  
 1   all_awardings                  5735 non-null   object 
 2   allow_live_comments            5735 non-null   bool   
 3   author                         5735 non-null   object 
 4   author_flair_css_class         5 non-null      object 
 5   author_flair_richtext          5679 non-null   object 
 6   author_flair_text              52 non-null     object 
 7   author_flair_type              5679 non-null   object 
 8   author_fullname                5679 non-null   object 
 9   author_patreon_flair           5679 non-null   object 
 10  author_premium                 5679 non-null   object 
 11  awarders                       5735 non-null   object 
 12  can_mod_post                   5735 non-null   b

In [50]:
#check column for usefullness

joined_df.subreddit_id.value_counts()

t5_34em3    2968
t5_3enp4    2767
Name: subreddit_id, dtype: int64

<b>observation</b>: subreddit column only show two values, therfore not meaningful

In [51]:
#shown distribution of number of comments 
joined_df.num_comments.value_counts().sort_index(ascending=False)

354       1
275       1
247       1
221       1
185       1
       ... 
4       436
3       483
2       716
1       687
0      1482
Name: num_comments, Length: 113, dtype: int64

In [52]:
#check to see if url column and full link column returns same value

print(joined_df[joined_df.num_comments==354]['url'].values)
print('\n')
print(joined_df[joined_df.num_comments==354]['full_link'].values)

['https://www.reddit.com/r/googlehome/comments/lb6ahl/post_your_comments_here_on_the_device_not_yet_set/']


['https://www.reddit.com/r/googlehome/comments/lb6ahl/post_your_comments_here_on_the_device_not_yet_set/']


Features that will be selected for futher exploration are stated in the table below.

| Features to keep:      	| Reason:                                            	|
|------------------------	|----------------------------------------------------	|
| subreddit              	| splits subreddit into r/GoogleHome or r/AmazonEcho 	|
| author                 	| to view distribution of post per author            	|
| full_link                 | contains link to reddit post                       	|
| num_comments           	| to view distribution of comments per post          	|
| url_overridden_by_dest 	| contains url used in a post                        	|
| selftext               	| contains text body of post                         	|
| title                  	| contains title of post                             	|
| link_flair_text           | cointains type of post                                |



In [53]:
#created df with selected features

joined_df = joined_df[['subreddit','author','full_link','num_comments','url_overridden_by_dest','selftext','title','link_flair_text']]

print(f'Dataframe with selected feature shape: {joined_df.shape}')

Dataframe with selected feature shape: (5735, 8)


In [54]:
#show columns and null values of new df
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   subreddit               5735 non-null   object
 1   author                  5735 non-null   object
 2   full_link               5735 non-null   object
 3   num_comments            5735 non-null   int64 
 4   url_overridden_by_dest  987 non-null    object
 5   selftext                4631 non-null   object
 6   title                   5735 non-null   object
 7   link_flair_text         3197 non-null   object
dtypes: int64(1), object(7)
memory usage: 358.6+ KB


Null values in selftext, as discuss earlier in subsection [investigate on documents with missing selftext](#investigate-on-documents-with-missing-selftext), these are posts that have titles that could be in the form of question or announcement, accompanied by a link or image instead of text. 

These titles therefore are sufficient enough to encapsulate the overall sentiment or message of their post, and therefore as a form of feature engineering, post with titles only and no selftext, will have their titles be copied into the selftext column to replace the null values. After which, the dataframe is ready to be exported as clean data for futher exploration.

In [55]:
#check for self text value_counts
joined_df.selftext.value_counts()

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  238
[deleted]                                                                                                                                                                                                                                                                                                                                                                                                

In [56]:
"""write for loop function to replace empty values in selftext column
with values from title column'"""

for x in range(0,len(joined_df)):   #iterate through every row
    if pd.isnull(joined_df.iloc[x,5]) or joined_df.iloc[x,5] == '[removed]' or  joined_df.iloc[x,5] == '[deleted]':     #check to see if value is null,'[deleted]' or '[removed]' in selftext column
        joined_df.iloc[x,5] = joined_df.iloc[x,6]  #copy value in title column, replacing null value 
    else:
        pass

In [57]:
#show columns and null values of feature engineered df

joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   subreddit               5735 non-null   object
 1   author                  5735 non-null   object
 2   full_link               5735 non-null   object
 3   num_comments            5735 non-null   int64 
 4   url_overridden_by_dest  987 non-null    object
 5   selftext                5735 non-null   object
 6   title                   5735 non-null   object
 7   link_flair_text         3197 non-null   object
dtypes: int64(1), object(7)
memory usage: 358.6+ KB


In [58]:
#export dataset as clean data

joined_df.to_csv(r'../datasets/clean_data.csv',index=False)